In [56]:
from os import listdir
from os.path import join
from nltk.tokenize import WordPunctTokenizer
from collections import defaultdict, Counter
from tqdm import tqdm_notebook as tqdm
import csv

### Составление словаря

Для начала научимся проходить по всем текстам, имеющимся в новостном датасете, разбивать его на токены с учетом предложений. Составим словарь из топа слов по частоте встречаемости (возьмём $10000$)

In [1]:
TOP_N = 10000

Будем проходиться по всем файлам из директории

In [9]:
filenames = listdir('Lenta/texts')

In [10]:
len(filenames)

36446

In [20]:
vocab = Counter()
tokenizer = WordPunctTokenizer()

for filename in tqdm(filenames):
    full_path = join('Lenta/texts', filename)
    with open(full_path) as input_file:
        for line in input_file.readlines():
            tokens = tokenizer.tokenize(line.lower())
            for token in tokens:
                vocab[token] += 1

In [29]:
vocab = vocab.most_common(TOP_N)

In [41]:
ind_to_word = [word for word, cnt in vocab]
word_to_ind = dict([(word, i) for i, word in enumerate(ind_to_word)])

### Загрузим фикслист гиперонимов

In [90]:
hyponyms_dict = defaultdict(list)
hypernyms_dict = defaultdict(list)

In [91]:
def add_pair(hyponym, hypernym):
    hyponyms_dict[hypernym].append(hyponym)
    hypernyms_dict[hyponym].append(hypernym)

In [92]:
with open('all_ru_pairs_ruwikt20160210_parsed.txt') as tsv_file:
    tsv_reader = csv.reader(tsv_file, delimiter='\t')
    for line in tsv_reader:
        if line[-1] == 'hyponyms':
            add_pair(line[2], line[1])
        elif line[-1] == 'hypernyms':
            add_pair(line[1], line[2])

In [93]:
with open('pairs-isas-aa.csv') as tsv_file:
    tsv_reader = csv.reader(tsv_file, delimiter='\t')
    next(tsv_reader, None)
    for line in tsv_reader:
        if int(line[2]) >= 5:
            add_pair(line[0], line[1])

In [94]:
hyponyms_dict['русский']

['врэвакуант',
 'древнерусский',
 'кацап',
 'кацапский',
 'коломнитин',
 'коломнянин',
 'новгородский',
 'псковский',
 'московский',
 'новгородский',
 'псковский',
 'рязанский',
 'скобарь']

In [112]:
hypernyms_dict['у']

['глаза',
 'лицо',
 'жизнь',
 'внешность',
 'волосы',
 'характер',
 'работа',
 'талант',
 'тело',
 'опыт',
 'голова',
 'рука',
 'любовь',
 'фигура',
 'ноги',
 'способность',
 'голос',
 'внешностью',
 'сердце',
 'кожа',
 'рана',
 'рожа',
 'имя',
 'состояние',
 'случай',
 'жизни',
 'душа',
 'мозг',
 'сила',
 'репутация',
 'глазами',
 'характером',
 'лицом',
 'ум',
 'цвет',
 'положение',
 'лица',
 'работе',
 'проблема',
 'деньги',
 'оружие',
 'память',
 'дар',
 'машина',
 'отношение',
 'взгляд',
 'красота',
 'здоровье',
 'нос',
 'опыта',
 'глаз',
 'дом',
 'семья',
 'одежда',
 'цвета',
 'форма',
 'рожей',
 'настроение',
 'платье',
 'связь',
 'болезнь',
 'количество',
 'степень',
 'способностями',
 'репутацией',
 'жена',
 'грудь',
 'отец',
 'фигурой',
 'темперамент',
 'степени',
 'вкус',
 'мать',
 'друг',
 'именем',
 'кровь',
 'руки',
 'ногами',
 'чувство',
 'проблем',
 'личико',
 'нервы',
 'воображение',
 'родители',
 'силы',
 'муж',
 'образование',
 'ситуация',
 'опытом',
 'власть',
 'сво

In [ ]:
# pymorphy - lemmatizer

### Гиперонимия в одном предложении

Напишем функцию, которая будет принимать предложения и детектировать в нем гиперонимию, если она есть. Тогда, если она есть, будет отмечать слова, где она встретилась

In [107]:
def find_sentence_hypernymy(sentence, tokenizer=None):
    if tokenizer is None:
        tokenizer = WordPunctTokenizer()
    tokens = tokenizer.tokenize(sentence.lower())
    hyper_ind = None
    hypo_ind = None
    for i, hypo_cand in enumerate(tokens):
        for j, hyper_cand in enumerate(tokens):
            if hypo_cand in hyponyms_dict[hyper_cand]:
                if len(hypo_cand) > 1 and len(hyper_cand) > 1:
                    hyper_ind = j
                    hypo_ind = i
                    break
    if hyper_ind is not None:
        sent_tokens = []
        for i, token in enumerate(tokens):
            if i == hypo_ind:
                token = "__" + token + "__"
            elif i == hyper_ind:
                token = "**" + token + "**"
            sent_tokens.append(token)
        print(' '.join(sent_tokens))
        print()

In [108]:
find_sentence_hypernymy('Московский уровень жизни выше, чем средний российский.')

Теперь пройдемся по имеющемуся корпусу и попробуем найти подобные предложения в новостях

In [3]:
import sys
sys.stdout = open('Sentences', 'w')

for filename in tqdm(filenames):
    full_path = join('Lenta/texts', filename)
    with open(full_path) as input_file:
        data = input_file.read().replace('\n', ' ')
        sentences = data.split('.')
        for sent in sentences:
            find_sentence_hypernymy(sent, tokenizer)

NameError: name 'tqdm' is not defined